# GraphRAG MCP Server Testing

This notebook tests the GraphRAG MCP tools directly and via the HTTP server.

**Prerequisites:**
1. Knowledge graph indexed: `poetry run python -m core.index`
2. Azure OpenAI configured in `.env`

**Optional (for HTTP tests):**

3. MCP server running: `poetry run python run_mcp_server.py`

In [ ]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent))

# Load environment variables
load_dotenv()

# Verify configuration
print("Configuration Check:")
print(f"   Azure OpenAI Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"   Chat Deployment: {os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT', 'gpt-4o')}")
print(f"   Embedding Deployment: {os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT', 'text-embedding-3-small')}")

In [ ]:
import logging

# ---------------------------------------------------------------------------
# Suppress noisy INFO logs from LiteLLM and GraphRAG internals.
#
# LiteLLM (used internally by GraphRAG) logs every Azure OpenAI API call at
# INFO level. Global search triggers 20+ parallel LLM calls, producing 40+
# log lines per query. GraphRAG also attaches a RichHandler that emits styled
# HTML fragments captured by Jupyter as text/html outputs, inflating the
# .ipynb file size without adding user value.
#
# We keep WARNING level so that actionable messages (e.g. token-limit
# warnings from mixed_context.py) are still visible for traceability.
# ---------------------------------------------------------------------------

logging.basicConfig(level=logging.WARNING)

# LiteLLM — main noise source (INFO on every completion call)
logging.getLogger("litellm").setLevel(logging.WARNING)

# GraphRAG internals — keeps token-limit WARNINGs, removes routine INFO
logging.getLogger("graphrag").setLevel(logging.WARNING)

# httpx / httpcore — suppress HTTP-level request chatter
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)

# OpenAI SDK — suppress retry/connection INFO
logging.getLogger("openai").setLevel(logging.WARNING)

print("\u2705 Logging configured \u2014 noisy INFO logs suppressed (WARNING+ preserved)")

## 1. Test MCP Server Connection (optional)

Only needed if you started the MCP server with `poetry run python run_mcp_server.py`.

In [ ]:
import httpx

# Test MCP server health
async def test_mcp_server():
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get("http://localhost:8011/health", timeout=5.0)
            print(f"✅ MCP Server Status: {response.status_code}")
            print(f"   Response: {response.text}")
    except httpx.ConnectError:
        print("❌ MCP Server not running!")
        print("   Start it with: poetry run python run_mcp_server.py")
    except Exception as e:
        print(f"❌ Error: {e}")

await test_mcp_server()

## 2. Test Entity Query Tool

List and search entities directly (no server needed).

In [ ]:
from mcp_server.tools.entity_query import entity_query_tool

# List entities by type
result = await entity_query_tool(entity_type="person", limit=5)

if "error" in result:
    print(f"Error: {result['error']}")
else:
    print(f"Found {result['total_found']} person entities (showing {result['returned']}):\n")
    for entity in result["entities"]:
        print(f"  - {entity['name']} ({entity['type']})")

# Search by name
print(f"\nSearching for 'Emily Harrison':")
result = await entity_query_tool(entity_name="Emily Harrison")
if "error" not in result:
    for entity in result["entities"]:
        print(f"  - {entity['name']}: {entity['description'][:100]}...")

## 3. Test Local Search (Entity-Focused)

In [ ]:
from mcp_server.tools.local_search import local_search_tool

queries = [
    "Who leads Project Alpha?",
    "Who resolved the GraphRAG index corruption incident?",
    "What technologies are used in Project Beta?",
]

for query in queries:
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print(f"{'='*70}\n")

    result = await local_search_tool(query)
    if "error" in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Answer: {result['answer'][:500]}...")
        print(f"\nContext: {result['context']}")

## 4. Test Global Search (Thematic)

**Note:** Global search uses map-reduce across all communities, making many parallel LLM calls. May be slow or hit rate limits.

In [ ]:
from mcp_server.tools.global_search import global_search_tool

query = "What are the main projects at TechVenture Inc?"
print(f"Query: {query}\n")

result = await global_search_tool(query)
if "error" in result:
    print(f"Error: {result['error']}")
    if "details" in result:
        print(f"Details: {result['details']}")
else:
    print(f"Answer:\n{result['answer']}")
    print(f"\nContext: {result['context']}")

## 5. Cross-Document Reasoning

In [ ]:
complex_queries = [
    "Who are the customers of Project Beta and what incidents affected them?",
    "What is the connection between David Kumar, Sophia Lee, and the GraphRAG incident?",
]

for query in complex_queries:
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print(f"{'='*70}\n")

    result = await local_search_tool(query)
    if "error" in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Answer: {result['answer'][:500]}...")

## Summary

This notebook demonstrates:

1. MCP Server connectivity check
2. Entity query tool (list/search entities)
3. Local search (entity-focused queries)
4. Global search (thematic/community queries)
5. Cross-document reasoning

**Testing via HTTP:** Use [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) for interactive server testing:
```bash
npx @modelcontextprotocol/inspector
```

**Next Steps:**
- Part 3: Supervisor Agent Pattern (agent-framework integration)